In [539]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup
import os
import pandas as pd

In [9]:
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [10]:
seasons = list(range(2021,2022))
seasons

[2021]

In [11]:
# Set up the browser
executable_path = {'executable_path':"C:\Program Files (x86)\msedgedriver.exe"}
browser = Browser('edge', **executable_path)

In [12]:
for season in seasons:
    # Visit the website for scraping
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    browser.visit(url)

    # Create a BeautifulSoup object
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    filter_div = soup.find('div', class_='filter')
    links = filter_div.find_all('a')
    urls = [link.get("href") for link in links]

In [13]:
table = soup.find('table', id='schedule')

links = table.find_all("a")
hrefs = [link.get('href') for link in links]
box_scores = [link for link in hrefs if link and "boxscore" in link and ".html" in link]
box_scores = [f"https://www.basketball-reference.com{score}" for score in box_scores]

In [14]:
box_score = box_scores[0]
browser.visit(box_score)

html=browser.html
soup = BeautifulSoup(html, 'html.parser')

In [15]:
score_table = soup.find('table', id='line_score')
tbody = score_table.find('tbody')
rows = tbody.find_all('tr')

line_score = []

for row in rows:
    # Get team name
    team = row.find('th', class_='center').text
    columns = row.find_all('td')
    
    # Create line score dictionary
    if(columns !=[]):
        total = columns[4].text
    
    # Create dictionary for dataframe later
    line_score_dict = { "team": team,
                        "total": total
    }
    
    # Add dictionary to array
    line_score.append(line_score_dict)

In [590]:
# Create Data frame
score_df = pd.DataFrame(line_score)
score_df

,team,total
0,GSW,99
1,BRK,125


In [431]:
# Get basic and advanced box scores
teams = []
for i in range(len(line_score)):
    teams.append(line_score[i]['team'])

In [634]:
stats = ['basic', 'advanced']
basic = []
advanced = []

for team in teams:
    for stat in stats:
        stat_table = soup.find('table', id=f"box-{team}-game-{stat}")
        table_data = []
        if stat_table:
            for row in stat_table.find_all('tr'):
                row_data = [cell.get_text() for cell in row.find_all(['th', 'td'])]
                table_data.append(row_data)
                
            # Store table data on the type of stat team
            if stat == 'basic':
                basic.extend(table_data)
            elif stat == 'advanced':
                advanced.extend(table_data)

In [635]:
basic = pd.DataFrame(basic)
basic

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,,Basic Box Score Stats,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,Starters,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
2,Andrew Wiggins,31:14,4,16,.250,2,6,.333,3,4,...,0,2,2,1,0,1,4,4,13,-28
3,Stephen Curry,30:19,7,21,.333,2,10,.200,4,4,...,3,1,4,10,2,0,3,1,20,-23
4,Kelly Oubre Jr.,25:39,3,14,.214,0,6,.000,0,0,...,4,3,7,2,1,2,3,1,6,-28
5,James Wiseman,24:17,7,13,.538,1,1,1.000,4,8,...,1,5,6,0,2,0,1,2,19,-10
6,Eric Paschall,21:33,2,6,.333,1,1,1.000,1,2,...,1,0,1,0,0,1,0,1,6,-28
7,Reserves,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
8,Brad Wanamaker,21:40,0,2,.000,0,1,.000,3,4,...,0,1,1,3,0,0,1,2,3,+1
9,Jordan Poole,17:55,1,4,.250,0,1,.000,0,0,...,0,2,2,3,0,0,2,2,2,-3


In [623]:
stats = ['basic', 'advanced']
home_team_tables = {'basic': [], 'advanced': []}
away_team_tables = {'basic': [], 'advanced': []}

for team in teams:
    for stat in stats:
        stat_table = soup.find('table', id=f"box-{team}-game-{stat}")
        table_data = []
        if stat_table:
            for row in stat_table.find_all('tr'):
                row_data = [cell.get_text() for cell in row.find_all(['th', 'td'])]
                table_data.append(row_data)
                
        # Store table data on the type of stat team
        if team == teams[0]:
            home_team_tables[stat].extend(table_data)
        elif team == teams[1]:
            away_team_tables[stat].extend(table_data)

In [606]:
home_team_tables['basic']

[['', 'Basic Box Score Stats'],
 ['Starters',
  'MP',
  'FG',
  'FGA',
  'FG%',
  '3P',
  '3PA',
  '3P%',
  'FT',
  'FTA',
  'FT%',
  'ORB',
  'DRB',
  'TRB',
  'AST',
  'STL',
  'BLK',
  'TOV',
  'PF',
  'PTS',
  '+/-'],
 ['Andrew Wiggins',
  '31:14',
  '4',
  '16',
  '.250',
  '2',
  '6',
  '.333',
  '3',
  '4',
  '.750',
  '0',
  '2',
  '2',
  '1',
  '0',
  '1',
  '4',
  '4',
  '13',
  '-28'],
 ['Stephen Curry',
  '30:19',
  '7',
  '21',
  '.333',
  '2',
  '10',
  '.200',
  '4',
  '4',
  '1.000',
  '3',
  '1',
  '4',
  '10',
  '2',
  '0',
  '3',
  '1',
  '20',
  '-23'],
 ['Kelly Oubre Jr.',
  '25:39',
  '3',
  '14',
  '.214',
  '0',
  '6',
  '.000',
  '0',
  '0',
  '',
  '4',
  '3',
  '7',
  '2',
  '1',
  '2',
  '3',
  '1',
  '6',
  '-28'],
 ['James Wiseman',
  '24:17',
  '7',
  '13',
  '.538',
  '1',
  '1',
  '1.000',
  '4',
  '8',
  '.500',
  '1',
  '5',
  '6',
  '0',
  '2',
  '0',
  '1',
  '2',
  '19',
  '-10'],
 ['Eric Paschall',
  '21:33',
  '2',
  '6',
  '.333',
  '1',
  '1',


In [325]:
home_advanced = home_advanced.apply(pd.to_numeric, errors="coerce")
home_basic = home_basic.apply(pd.to_numeric, errors="coerce")
away_advanced = away_advanced.apply(pd.to_numeric, errors="coerce")
away_basic = away_basic.apply(pd.to_numeric, errors="coerce")

In [657]:
home_advanced = pd.DataFrame(home_team_tables['advanced']).dropna().drop(index=7)
home_basic = pd.DataFrame(home_team_tables['basic']).dropna().drop(index=7)
away_advanced = pd.DataFrame(away_team_tables['advanced']).dropna().drop(index=7)
away_basic = pd.DataFrame(away_team_tables['basic']).dropna().drop(index=7)

# Use first row as headers
advanced_headers = home_advanced.iloc[0]
basic_headers = home_basic.iloc[0]


# Set headers
home_advanced.rename(columns=advanced_headers, inplace=True)
away_advanced.rename(columns=advanced_headers, inplace=True)

home_basic.rename(columns=basic_headers, inplace=True)
away_basic.rename(columns=basic_headers, inplace=True)

home_advanced = home_advanced.drop(index=1).reset_index(drop=True)
away_advanced = away_advanced.drop(index=1).reset_index(drop=True)
home_basic = home_basic.drop(index=1).reset_index(drop=True)
away_basic = away_basic.drop(index=1).reset_index(drop=True)

# Use Players as index
home_advanced = home_advanced.set_index('Starters')
away_advanced = away_advanced.set_index('Starters')
home_basic = home_basic.set_index('Starters')
away_basic = away_basic.set_index('Starters')

In [658]:
away_basic.head()

,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
Starters,,,,,,,,,,,,,,,,,,,,
Kyrie Irving,25:18,10,16,.625,4,7,.571,2,2,1.000,1,3,4,4,0,0,1,3,26,+32
Kevin Durant,24:56,7,16,.438,1,2,.500,7,7,1.000,1,4,5,3,3,1,1,3,22,+26
Joe Harris,20:46,4,8,.500,2,5,.400,0,0,,2,5,7,2,0,1,2,2,10,+21
Spencer Dinwiddie,19:57,2,6,.333,1,3,.333,4,4,1.000,0,5,5,4,0,0,3,1,9,+24
DeAndre Jordan,17:03,2,3,.667,0,0,,0,2,.000,2,9,11,1,1,1,3,2,4,+21


In [661]:
summaries = []
base_cols = None

for team in teams:
    # Calculate totals for the current team
    totals = pd.concat([home_basic.iloc[-1,:], home_advanced.iloc[-1,:]])
    totals.index = totals.index.str.lower()
    
    # Calculate maxes for the current team
    maxes = pd.concat([home_basic.iloc[:-1,:].max(), home_advanced.iloc[:-1,:].max()])
    maxes.index = maxes.index.str.lower() + "_max"
    
    # Combine totals and maximums into a summary dataframe
    summary = pd.concat([totals, maxes])

    if base_cols is None:
        base_cols = list(summary.index.drop_duplicates(keep='first'))
        base_cols = [b for b in base_cols if "bpm" not in b]

    summary = summary[base_cols]
    summaries.append(summary)

summary = pd.concat(summaries, axis=1).T

In [662]:
summary

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,orb%_max,drb%_max,trb%_max,ast%_max,stl%_max,blk%_max,tov%_max,usg%_max,ortg_max,drtg_max
0,240,240,37,99,.374,10,33,.303,15,23,...,8.3,6.5,8.2,61.1,3.7,6.6,50.0,8.3,99,120
1,240,240,37,99,.374,10,33,.303,15,23,...,8.3,6.5,8.2,61.1,3.7,6.6,50.0,8.3,99,120
